In [1]:
! pip install -q python-dotenv
! pip install -q neo4j
! pip install -q langchain
! pip install -q langchain-openai
! pip install -q tiktoken


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import dotenv
import os

dotenv.load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("URL")
os.environ["NEO4J_USERNAME"]= os.getenv("USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("PASSWORD2")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAIKEY")

In [3]:
from langchain_community.graphs import Neo4jGraph
import sys
sys.path.append('../utils')  
from helper import write_chunks_to_df


In [4]:
graphDB = Neo4jGraph()

In [6]:
#graphDB.query("SHOW INDEXES")

In [7]:
#
# Initialize Semantic Vector Index
#
# DOCS: https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/vector-indexes/
#

vector_index_query=""" 
CALL db.index.vector.createNodeIndex(
  'accreditation_index',
  'Chunk',
  'embedding',
   1536,
  'cosine'
)
"""

graphDB.query(vector_index_query)

[]

In [8]:
graphDB.query("SHOW INDEXES")

[{'id': 5,
  'name': 'accreditation_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 1,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 17, 5, 29, 23, 62000000, tzinfo=<UTC>),
  'readCount': 802},
 {'id': 2,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 3,
  'name': 'unique_node',
  'state': 'ONLIN

### Generate Vector Embeddings

In [9]:
#
#  Query All Chunks without embedding
#

all_chunks_query = """ 
MATCH (c:Chunk) 
WHERE c.embedding IS null OR c.embedding = 0
RETURN c
"""


# attach embedding to chunk.embedding
vector_to_chunk_query = """ 
MATCH (c:Chunk {UUID: $UUID})
SET c.embedding = $vector
"""

In [10]:
result = graphDB.query(all_chunks_query)

In [11]:
#
# Generate Vectors for Chunk.text and update Chunk.embedding
#

chunk_dataframe = write_chunks_to_df(result)

In [13]:
#chunk_dataframe.head()
chunk_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      208 non-null    object
 1   nodeType  208 non-null    object
 2   UUID      208 non-null    object
dtypes: object(3)
memory usage: 5.0+ KB


In [14]:
#
# Chunk embedding function
# 
# DOCS: https://platform.openai.com/docs/guides/embeddings/use-cases
#


from openai import OpenAI

client =OpenAI()

#MODEL =  "text-embedding-3-small"
MODEL = "text-embedding-ada-002"

def get_embedding(text, model = MODEL):

    text = text.replace("\n"," ")

    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [15]:
chunk_dataframe['vector'] = chunk_dataframe['text'].apply(lambda x:get_embedding(x))

In [16]:
chunk_dataframe.head()

,text,nodeType,UUID,vector
0,standard 1 strategic planning1.1 school mainta...,DATACHUNK,fb8549d4-efa5-4c55-a7d3-87522a416b98,"[0.002811488462612033, 0.0041966927237808704, ..."
1,progress planned strategies expected outcomes ...,DATACHUNK,d6e18b0a-2aca-4de8-a294-e6160f186a0d,"[-0.01016693189740181, 0.0034439638257026672, ..."
2,1.1 maintenance strategic plan school mission ...,DATACHUNK,a67bc7ec-d5f2-44c4-a6e0-6d1e94af611b,"[-0.008482272736728191, -0.010247484780848026,..."
3,plan . strategic plan developed refined engage...,DATACHUNK,e143576e-9893-4999-a5f7-a898d3388193,"[0.0022559347562491894, 0.0026812339201569557,..."
4,serve including level degree programs school o...,DATACHUNK,eecdf569-2c10-4574-ab4a-a8f6117a0b6d,"[0.007008944638073444, 0.008684996515512466, -..."


In [17]:
for i, row in chunk_dataframe.iterrows():

    graphDB.query(vector_to_chunk_query, 
                params ={
                    'UUID':row['UUID'],
                    'vector':row['vector']
                })

### Queries

In [21]:
semantic_index_query = """

CALL db.index.vector.queryNodes('accreditation_index', 2, $inputVector)
YIELD node AS responseNode, score

RETURN responseNode.text, score 
"""

In [31]:
query_text = "what are aacsb standards"
query_vector = get_embedding(query_text)
query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })

In [32]:
query_result

[{'responseNode.text': '3.1 faculty sufficiency school adopts applies criteria documenting faculty members participating supporting consistent mission . school adapt guidance particular situation mission developing implementing criteria indicate school meeting spirit intent standard . criteria address activities required attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed',
  'score': 0.9091740846633911},
 {'responseNode.text': 'attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed reflect focus continuous improvement . normally participating faculty members deliver least 75 percent school teaching globally i.e . across entire accredited unit participating faculty members deliver least 60 percent teaching w

In [26]:
#
# Standard QA
#
from langchain_community.vectorstores import Neo4jVector

VECTOR_INDEX_NAME = 'accreditiation-index'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY='embedding'


In [ ]:
#https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever/


In [28]:
# Query the knowledge graph in a RAG application
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate



graphDB.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    graph=graphDB,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    validate_cypher=True, # Validate relationship directions
    verbose=True
)

In [30]:
#cypher_chain.invoke({"query": "How many learning objectives are assessed"})
#cypher_chain.invoke({"query": "Which student learning goals were identified"}) # does not know
#cypher_chain.invoke({"query": "What are the descriptions of the Learning goal"}) ## does not know , see still Learning goal example
cypher_chain.invoke({"query": "Tell me about AACSB standards"}) ## does not know , see still Learning goal example




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Standard {source: 'AACSB'}) RETURN s
Full Context:
[]

> Finished chain.


{'query': 'Tell me about AACSB standards',
 'result': "I don't know the answer."}

Out of the box LLM genreate cypher queries are not producing any results, OR are producing errors. @TODO, provide some examples to the mode an oppopriate queries based on the schema. 

In [34]:
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
chain  = RetrievalQAWithSourcesChain.from_chain_type(

)

In [54]:
#
#  LangChain Docs: https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever/
#     



In [116]:

import os
import re
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)


def extract_list(llm_result):
    content = llm_result.content
    return content.split("\\n")

def generate_multi_question(question):

    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to generate three 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by new line. 
    
    # Format Rules
    DO NOT NUMBER THE LIST
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)
     



    return {'original_query': question, 'generated_query_list': query_list}

In [118]:
from typing import List, Dict





def generate_multi_question_aacsb(question: str) -> Dict[str, any]:
    """
    For queries relating to the AACSB standards, generates list three different versions
    of the user's input query. Used downstream for multiquery retrieval.

    Args:
        question (str): The user's input query.

    Returns:
        Dict[str, any]: A dictionary containing the original user question and the generated list of alternative 
        questions. Keys include 'original_query' for the original question and 'generated_query_list' for the list 
        of alternative questions.

    """


    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to generate three 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search, by providing precise questions.
    Provide these alternative questions separated by new line. 

    # Important Context: AACSB
    The questions you generate are directly related to extracting useful information on the 
    AACSB accrediation standards. The questions you generate will be used as vector database index
    queries that contain information on:

    - formal AACSB descriptions
    - documentation that supports each standard 
    - basis for evaluation of the standards 
    - relevent definitions of terms used in the standard descriptions.

    The AACSB Website provides the following summary of their work:
    AACSB accreditation is known, worldwide, as the longest-standing, most recognized form of 
    specialized accreditation that an institution and its business programs can earn. 
    Accreditation is a voluntary, nongovernmental process that includes a rigorous external review 
    of a school's mission, faculty qualifications, curricula, and ability to provide the highest-quality programs.
    
    # Format Rules
    DO NOT NUMBER THE LIST
    DO NOT ANSWER THE QUESTION
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)  



    return {'original_query': question, 'generated_query_list': query_list, 'cat':'AACSB'}

In [120]:
from typing import List, Dict


def generate_multi_question_institution(question: str) -> Dict[str, any]:
    """
    For school specific, or academic institution specific, queries. Generates list three different versions
    of the user's input query. Used downstream for multiquery retrieval.

    Args:
        question (str): The user's input query.

    Returns:
        Dict[str, any]: A dictionary containing the original user question and the generated list of alternative 
        questions. Keys include 'original_query' for the original question and 'generated_query_list' for the list 
        of alternative questions.
    """


    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to generate three 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search, by providing precise questions.
    Provide these alternative questions separated by new line. 

    # Important Context: Academic Institution
    The questions you generate are directly related to extracting useful information about a School
    of Business.  The questions you generate will be used as vector database index
    queries that contain information on:

    - Strategic Plan, Mission and Fiscal Resources
    - Academic Departments in the School of Business inluding not limited to : Accounting, Marketing, Management, Finance, Entreprenuership
    - Student Services and Student Organizations 
    - Program Goals, Learning Objectives and Curriculum Assessment
    - Continuous Improvement


    # Format Rules
    DO NOT NUMBER THE LIST
    DO NOT ANSWER THE QUESTION
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)  



    return {'original_query': question, 'generated_query_list': query_list, 'cat':'INSTITUTION'}

In [121]:
from typing import List, Dict


def generate_sub_questions_hybrid(question: str) -> Dict[str, any]:
    """
    For queries relating to both AACSB accreditation AND Academic Institution (School) information .
    Generates two distinct sub questions based on the user's input query. Used downstream for multiquery retrieval.

    Args:
        question (str): The user's input query.

    Returns:
        Dict[str, any]: A dictionary containing the original user question and the generated list of alternative 
        questions. Keys include 'original_query' for the original question and 'generated_query_list' for the list 
        of alternative questions.
    """


    system_prompt = f""" 

    # Instruction
    You are an AI language model assistant. Your task is to evalute the users input query and 
    divide the query into two and ONLY TWO sub questions. The first sub questions should address the portion
    of the user query that relates to the AACSB Standards, the second subquestion should relate the institution
    specific portion of the query. Your overall objective is to break down the complex user query into the 
    two distinct sub questions.  Provide these alternative questions separated by new line. 

    # Important Context: Sub 
    
    ## 1. AACSB Standard sub question:
    AACSB sub question may related to accreditation content such as:

    - formal AACSB descriptions
    - documentation that supports each standard 
    - basis for evaluation of the standards 
    - relevent definitions of terms used in the standard descriptions.

    The AACSB Website provides the following summary of their work:
    AACSB accreditation is known, worldwide, as the longest-standing, most recognized form of 
    specialized accreditation that an institution and its business programs can earn. 
    Accreditation is a voluntary, nongovernmental process that includes a rigorous external review 
    of a school's mission, faculty qualifications, curricula, and ability to provide the highest-quality programs.


    ## 2. Academic Instiution sub question:
    Academic Instiution sub question may relate to School of Business content such as:

    - Strategic Plan, Mission and Fiscal Resources
    - Academic Departments in the School of Business inluding not limited to : Accounting, Marketing, Management, Finance, Entreprenuership
    - Student Services and Student Organizations 
    - Program Goals, Learning Objectives and Curriculum Assessment
    - Continuous Improvement


    # Format Rules
    DO NOT NUMBER THE LIST
    DO NOT ANSWER THE QUESTION
    Original question: {question}
    """ 
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
    result = llm.invoke(system_prompt)
    query_list = extract_list(result)  



    return {'original_query': question, 'generated_query_list': query_list, 'cat':'HYBRID'}

In [83]:
test_query = "how should I prepare for extended travel"

result = generate_multi_question(test_query)

In [84]:
for r in result['generated_query_list']:
    print (r)

- What are some tips for preparing for an extended travel?
- Can you provide advice on getting ready for extended travel?
- What steps should I take to prepare for a long trip?
- How can I best get myself ready for an extended travel?
- Are there any recommendations for preparing for a lengthy journey?


In [93]:

test_query = "Our accountind department recently  updated the curriculum to include carbon footprint, does this reflect the sustainability standard "
test_hybrid_query = generate_sub_questions_hybrid(test_query)

In [94]:
test_hybrid_query

{'original_query': 'Our accountind department recently  updated the curriculum to include carbon footprint, does this reflect the sustainability standard ',
 'generated_query_list': ['Alternative questions:\n1. What are the formal AACSB descriptions and documentation that support the sustainability standard?\n2. How does the updated curriculum in our accounting department align with the sustainability standard?'],
 'cat': 'HYBRID'}

In [122]:
from openai import OpenAI
import json

client = OpenAI()



def run_conversation(user_query:str)->str:
    """
    Run a conversation with OpenAI's language model, providing the user query and available functions to the model.

    Args:
        user_query (str): The user's input query.

    Returns:
        str: The response from the language model.

    Step 1: Send the conversation and available functions to the model.
        - Each function is described with its name, description, and parameters.
        - Three functions are available:
            1. generate_multi_question_aacsb: Generates three different versions of the user's input query
               related to AACSB standards.
            2. generate_multi_question_institution: Generates three different versions of the user's input query
               for school-specific or academic institution-specific queries.
            3. generate_sub_questions_hybrid: Generates two distinct sub-questions based on the user's input query,
               relating to both AACSB accreditation and academic institution (school) information.

    Returns the response from the language model, specifically the first choice message from the available choices.
    """


    messages = [{"role": "user", "content": user_query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "generate_multi_question_aacsb",
                "description": "For queries relating to the AACSB standards, generates list three different versions of the user's input query. Used downstream for multiquery retrieval.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Input user query",
                        }
                    },
                    "required": ["question"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "generate_multi_question_institution",
                "description": "For school specific, or academic institution specific, queries. Generates list three different versions of the user's input query. Used downstream for multiquery retrieval.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Input user query",
                        }
                    },
                    "required": ["question"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "generate_sub_questions_hybrid",
                "description": "For queries relating to both AACSB accreditation AND Academic Institution (School) information. Generates two distinct sub questions based on the user's input query. Used downstream for multiquery retrieval.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Input user query",
                        }
                    },
                    "required": ["question"],
                },
            },
        }

    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default
    )
    return response.choices[0].message
    


In [102]:
input_query = "AACSB standards on sustainability"
result = run_conversation(input_query)

In [108]:
# import json

# function_name =  result.tool_calls[0].function.name
# function_args = json.loads(result.tool_calls[0].function.arguments)['question']
# print(f"{function_name} , {function_args}, ")

generate_multi_question_aacsb , AACSB standards on sustainability, 


In [109]:
# function = globals()[function_name]
# print(function(function_args))

{'original_query': 'AACSB standards on sustainability', 'generated_query_list': ['1. What are the AACSB accrediation standards on sustainability?\n2. How does AACSB define sustainability in their accrediation standards?\n3. Can you provide information on the AACSB standards related to sustainability?']}


In [123]:
import json

def execute_route_function(conv_result: any)->any:
    """
    Execute a route function based on the provided conversation result.

    Args:
        conv_result (any): The conversation result containing information about the function to execute.

    Returns:
        any: The result of executing the route function.

    Extracts the name and arguments of the route function from the conversation result and then
    dynamically executes the function using the extracted information.

    Note:
        - The function_name and function_args are extracted from the tool_calls attribute of the conversation result.
        - The function_name is used to retrieve the actual function from the global namespace.
        - The function_args are passed as arguments to the retrieved function.
    """

    function_name =  result.tool_calls[0].function.name
    function_args = json.loads(result.tool_calls[0].function.arguments)['question']
    function = globals()[function_name]
    return function(function_args)





In [124]:
def query_routing_pipeline(user_query):
    """
    Run a conversation with OpenAI's language model using the provided user query and then execute the route function.

    Args:
        user_query (str): The user's input query.

    Returns:
        dict: The result of executing the route function.

    This function serves as a pipeline for querying and routing based on the user's input.
    It first runs a conversation with OpenAI's language model using the provided user query.
    The result of the conversation is then passed to the execute_route_function to determine and execute the appropriate route function.
    The result of executing the route function is returned as a dictionary.
    """
    result = run_conversation(user_query)
    return execute_route_function(result) #dict



In [126]:

test_query_in = "AACSB standards on sustainability"
pipeline_result = query_routing_pipeline(test_query_in)

In [127]:
pipeline_result

{'original_query': 'AACSB standards on sustainability',
 'generated_query_list': ['1. What are the AACSB standards for sustainability?\n2. Can you provide information on the AACSB accreditation standards related to sustainability?\n3. How does the AACSB incorporate sustainability into its accreditation standards?'],
 'cat': 'AACSB'}

In [138]:
def retriever(query_dict, cypher_chain=cypher_chain):

    context_data = []

    if query_dict['cat'] != 'HYBRID':
        # multiqueries -- case A
        # single cypher query for original query
        for q in query_dict['generated_query_list']:
            query_vector = get_embedding(q)
            query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })
            query_text = [text['responseNode.text'] for text in query_result]
            context_data.extend(query_text)
        
        cypher_result = cypher_chain.invoke({"query": query_dict['original_query']})

        if "I don't know the answer" not in cypher_result['result']:
            context_data.append(cypher_result['result'])
            

    
    else:
        # subqueries -- case B
        # cypher query for each subquery
        for q in query_dict[generated_query_list]:
            query_vector = get_embedding(q)
            query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })
            query_text = [text['responseNode.text'] for text in query_result]
            context_data.extend(query_text)

            cypher_result = cypher_chain.invoke({"query": q})

            if "I don't know the answer" not in cypher_result['result']:
                context_data.append(cypher_result['result'])
                




        




    #if not hybrid
    ### loop through multiqueries
    #### call embedding function on each and retrieve data from db
    #### append to  context data
    ### call cypher query, append results to cyher data

    #else:
    ## loop through sub queries
    ### call embedding and retrieve append resutls to context data
    ### call cypher, append results to context data

    return ', '.join(context_data) #return context data as a string


In [139]:
ret_result = retriever(pipeline_result)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Standard {title: "AACSB standards on sustainability"}) RETURN s
Full Context:
[]

> Finished chain.


In [142]:
ret_result

'3.1 faculty sufficiency school adopts applies criteria documenting faculty members participating supporting consistent mission . school adapt guidance particular situation mission developing implementing criteria indicate school meeting spirit intent standard . criteria address activities required attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed, attain participating supporting status depth breadth activities expected within typical aacsb accreditation review cycle maintain participating supporting status . criteria periodically reviewed reflect focus continuous improvement . normally participating faculty members deliver least 75 percent school teaching globally i.e . across entire accredited unit participating faculty members deliver least 60 percent teaching within discipline regardless whether school degree major concentration etc .'

In [144]:
def generator(original_query, context_string, model = "gpt-3.5-turbo-16k"):

    system_prompt = f"""
    # Instruction:
    You are an AI assistant generating a thorough and thoughtful response to a user query.
    You are to generate a response that answers their query solely based on the context provided below
    do not use any other outside information. The grounding context information has been retrieved from
    a database which is the absolute knowledge source. 

    # User Query:
    {original_query}

    # Grounding Context:
    {context_string}

    # Output Style
    Your tone should be professional. And your response should be detailed, as this information
    will be used to generate reports. 

    """


    llm = ChatOpenAI(model=model, temperature=0)
    result = llm.invoke(system_prompt)
    content = result.content
     


    return content

In [146]:
gen = generator(test_query_in, ret_result)

In [147]:
gen

"According to the AACSB standards on sustainability, schools are required to adopt and apply criteria that document faculty members' participation and support of the consistent mission. These criteria should be adapted to the specific situation and mission of the school, and should indicate how the school is meeting the spirit and intent of the standard. The criteria should address the activities required to attain participating and supporting status, both in terms of depth and breadth of activities. These activities are expected to be within the typical AACSB accreditation review cycle, and should be maintained to maintain participating and supporting status.\n\nThe criteria should be periodically reviewed to reflect a focus on continuous improvement. In terms of faculty involvement, it is expected that participating faculty members deliver at least 75 percent of the school's teaching globally, across the entire accredited unit. Additionally, participating faculty members should deliv